In [ ]:
import os
import sys
import re
import shutil
import time
import pickle
from collections import defaultdict
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers, Elasticsearch
from elasticsearch.helpers import bulk
from datetime import datetime
import elasticsearch
import datetime
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

COLORS = ["slategrey","darkorange","royalblue"]

def fetch_data(data_dir):

   
    if not os.path.isdir(data_dir):
        raise Exception("specified data dir does not exist")
    if not len(os.listdir(data_dir)) > 0:
        raise Exception("specified data dir does not contain any files")

    file_list = [os.path.join(data_dir,f) for f in os.listdir(data_dir) if re.search("\.json",f)]
    correct_columns = ["country", "customer_id", "day", "invoice", "month",
                       "price", "stream_id", "times_viewed", "year"]

    ##############################
    all_months = {}
    for file_name in file_list:
        data = pd.read_json(file_name)
        all_months[os.path.split(file_name)[-1]] = data

    #format data with correct column names 
    for f,data in all_months.items():
        cols = set(data.columns.tolist())
        if 'total_price' in cols:
            data.rename(columns={'total_price':'price'},inplace=True)
        if 'TimesViewed' in cols:
            data.rename(columns={'TimesViewed':'times_viewed'},inplace=True)
        if 'StreamID' in cols:
             data.rename(columns={'StreamID':'stream_id'},inplace=True)
        

        cols = data.columns.tolist()
        if sorted(cols) != correct_columns:
            raise Exception("columns name could not be matched to correct cols")

    #data concatenation 
    data = pd.concat(list(all_months.values()),sort=True)
    years,months,days = data['year'].values,data['month'].values,data['day'].values 
    dates = ["{}-{}-{}".format(years[i],str(months[i]).zfill(2),str(days[i]).zfill(2)) for i in range(data.shape[0])]
    data['invoice_date'] = np.array(dates,dtype='datetime64[D]')
    data['invoice'] = [re.sub("\D+","",i) for i in data['invoice'].values]
    
    ## sort by date and reset the index
    data.sort_values(by='invoice_date',inplace=True)
    data.reset_index(drop=True,inplace=True)
    
    return(data)


def convert_to_ts(data_orig, country=None):
   

    if country:
        if country not in np.unique(df_orig['country'].values):
            raise Excpetion("country not found")
    
        mask = data_orig['country'] == country
        data = data_orig[mask]
    else:
        data = data_orig
        
    ## use a date range to ensure all days are accounted for in the data
    invoice_dates = df['invoice_date'].values
    start_month = '{}-{}'.format(df['year'].values[0],str(data['month'].values[0]).zfill(2))
    stop_month = '{}-{}'.format(df['year'].values[-1],str(data['month'].values[-1]).zfill(2))
    data_dates = data['invoice_date'].values.astype('datetime64[D]')
    days = np.arange(start_month,stop_month,dtype='datetime64[D]')
    
    purchases = np.array([np.where(df_dates==day)[0].size for day in days])
    invoices = [np.unique(data[data_dates==day]['invoice'].values).size for day in days]
    streams = [np.unique(data[data_dates==day]['stream_id'].values).size for day in days]
    views =  [data[data_dates==day]['times_viewed'].values.sum() for day in days]
    revenue = [data[data_dates==day]['price'].values.sum() for day in days]
    year_month = ["-".join(re.split("-",str(day))[:2]) for day in days]

    data_time = pd.DataFrame({'date':days,
                            'purchases':purchases,
                            'unique_invoices':invoices,
                            'unique_streams':streams,
                            'total_views':views,
                            'year_month':year_month,
                            'revenue':revenue})
    return(data_time)


def fetch_ts(data_dir, clean=False):
   
    #convenience function to read in new data
    
  

    ts_data_dir = os.path.join(data_dir,"ts-data")
    
    if clean:
        shutil.rmtree(ts_data_dir)
    if not os.path.exists(ts_data_dir):
        os.mkdir(ts_data_dir)

         
    if len(os.listdir(ts_data_dir)) > 0:
        print("... loading ts data from files")
        return({re.sub("\.csv","",cf)[3:]:pd.read_csv(os.path.join(ts_data_dir,cf)) for cf in os.listdir(ts_data_dir)})

    ## get original data
    print("..... processing data")
    df = fetch_data(data_dir)

    ## find the top ten countries (wrt revenue)
    table = pd.pivot_table(df,index='country',values="price",aggfunc='sum')
    table.columns = ['total_revenue']
    table.sort_values(by='total_revenue',inplace=True,ascending=False)
    top_ten_countries =  np.array(list(table.index))[:10]

    file_list = [os.path.join(data_dir,f) for f in os.listdir(data_dir) if re.search("\.json",f)]
    countries = [os.path.join(data_dir,"ts-"+re.sub("\s+","_",c.lower()) + ".csv") for c in top_ten_countries]

    ## load the data
    dfs = {}
    dfs['all'] = convert_to_ts(df)
    for country in top_ten_countries:
        country_id = re.sub("\s+","_",country.lower())
        file_name = os.path.join(data_dir,"ts-"+ country_id + ".csv")
        dfs[country_id] = convert_to_ts(df,country=country)

    ## save the data as csvs    
    for key, item in dfs.items():
        item.to_csv(os.path.join(ts_data_dir,"ts-"+key+".csv"),index=False)
        
    return(dfs)

def engineer_features(df,training=True):
    """
    for any given day the target becomes the sum of the next days revenue
    for that day we engineer several features that help predict the summed revenue
    
    the 'training' flag will trim data that should not be used for training
    when set to false all data will be returned

    """

    ## extract dates
    dates = df['date'].values.copy()
    dates = dates.astype('datetime64[D]')

    ## engineer some features
    eng_features = defaultdict(list)
    previous =[7, 70]  #[7, 14, 21, 28, 35, 42, 49, 56, 63, 70]
    y = np.zeros(dates.size)
    for d,day in enumerate(dates):

        ## use windows in time back from a specific date
        for num in previous:
            current = np.datetime64(day, 'D') 
            prev = current - np.timedelta64(num, 'D')
            mask = np.in1d(dates, np.arange(prev,current,dtype='datetime64[D]'))
            eng_features["previous_{}".format(num)].append(df[mask]['revenue'].sum())

        ## get get the target revenue    
        plus_30 = current + np.timedelta64(30,'D')
        mask = np.in1d(dates, np.arange(current,plus_30,dtype='datetime64[D]'))
        y[d] = df[mask]['revenue'].sum()

        ## attempt to capture monthly trend with previous years data (if present)
        start_date = current - np.timedelta64(365,'D')
        stop_date = plus_30 - np.timedelta64(365,'D')
        mask = np.in1d(dates, np.arange(start_date,stop_date,dtype='datetime64[D]'))
        eng_features['previous_year'].append(df[mask]['revenue'].sum())

        ## add some non-revenue features
        minus_30 = current - np.timedelta64(30,'D')
        mask = np.in1d(dates, np.arange(minus_30,current,dtype='datetime64[D]'))
        eng_features['recent_invoices'].append(df[mask]['unique_invoices'].mean())
        eng_features['recent_views'].append(df[mask]['total_views'].mean())

    X = pd.DataFrame(eng_features)
    ## combine features in to df and remove rows with all zeros
    X.fillna(0,inplace=True)
    mask = X.sum(axis=1)>0
    X = X[mask]
    y = y[mask]
    dates = dates[mask]
    X.reset_index(drop=True, inplace=True)

    if training == True:
        ## remove the last 30 days (because the target is not reliable)
        mask = np.arange(X.shape[0]) < np.arange(X.shape[0])[-30]
        X = X[mask]
        y = y[mask]
        dates = dates[mask]
        X.reset_index(drop=True, inplace=True)
        


#    print(X)

#    dfplot1 = X[['previous_7']].copy()
#    dfplot2 = X[['previous_70']].copy()
#    dfplot3 = X[['previous_year']].copy()
    
#    fig = plt.figure(figsize=(10.0, 8.0))
#    ax = fig.add_subplot(111)
#    plt.plot(dfplot1, "r", label="pre7days")
#    plt.plot(dfplot2, "g", label="pre70days")
#    plt.plot(dfplot3, "b", label="pre1year")
#    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
#    plt.show()

    return(X,y,dates)


In [ ]:
import os
import numpy as np


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.covariance import EllipticEnvelope
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from application.cslib import fetch_ts, engineer_features



data_dir = os.path.join("data","cs-train")

ts_all = fetch_ts(data_dir,clean=False)

X,y,dates = engineer_features(ts_all['all'])
        
## Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)
pipe_lr = Pipeline([('scaling', StandardScaler()),('regr',LinearRegression())])
pipe_lr.fit(X_train, y_train)
                           
y_pred_base = pipe_lr.predict(X)



param_grid_rf = {
    'rf__criterion': ['mse','mae'],
    'rf__n_estimators': [10,15,20,25,50,100]
    }

pipe_rf = Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

grid = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline
plt.plot(dates, y,label="groundtruth") 
plt.plot(dates, y_pred_base,label="baseline") 
plt.plot(dates, y_pred,label="model") 
plt.legend()
plt.show()

In [ ]:
if __name__ == "__main__":

    run_start = time.time() 
    data_dir = os.path.join("c:\\","data_ml","cs-train")
    
    print("...reading data to store pd dataframe")    
    df_time = convert_to_ts(fetch_data(data_dir), country=None)
    print(df_time)
    
    m, s = divmod(time.time()-run_start,60)
    h, m = divmod(m, 60)
    print("")
    print("load time:", "%d:%02d:%02d"%(h, m, s))

...reading data to store pd dataframe
          date  purchases  unique_invoices  unique_streams  total_views  \
0   2017-11-01          0                0               0            0   
1   2017-11-02          0                0               0            0   
2   2017-11-03          0                0               0            0   
3   2017-11-04          0                0               0            0   
4   2017-11-05          0                0               0            0   
..         ...        ...              ...             ...          ...   
602 2019-06-26       1358               67             999         6420   
603 2019-06-27       1620               80             944         9435   
604 2019-06-28       1027               70             607         5539   
605 2019-06-29          0                0               0            0   
606 2019-06-30        602               27             423         2534   

    year_month  revenue  
0      2017-11     0.00  
1      2017-11     0.00  
2      2017-11     0.00  
3      2017-11     0.00  
4      2017-11     0.00  
..         ...      ...  
602    2019-06  4903.17  
603    2019-06  5499.38  
604    2019-06  3570.60  
605    2019-06     0.00  
606    2019-06  1793.98  

[607 rows x 7 columns]

load time: 0:00:16

In [ ]:
if __name__ == "__main__":

    run_start = time.time() 
    data_dir = os.path.join("c:\\","data_ml","cs-train")
    
    print("...fetching data to make csv")
    ts_all = fetch_ts(data_dir, clean=True)
    for key,item in ts_all.items():
        print(key,item.shape)
    
    m, s = divmod(time.time()-run_start,60)
    h, m = divmod(m, 60)
    print("")
    print("load time:", "%d:%02d:%02d"%(h, m, s))

...fetching data to make csv
..... processing data
all (607, 7)
united_kingdom (607, 7)
eire (607, 7)
germany (607, 7)
france (607, 7)
norway (577, 7)
spain (607, 7)
hong_kong (426, 7)
portugal (607, 7)
singapore (456, 7)
netherlands (607, 7)

load time: 0:00:51

In [ ]:
if __name__ == "__main__":

    run_start = time.time() 
    data_dir = os.path.join("c:\\","data_ml","cs-train")
    
    print("...making engineer features")
    engineer_features(df_time,training=True)
    
    m, s = divmod(time.time()-run_start,60)
    h, m = divmod(m, 60)
    print("load time:", "%d:%02d:%02d"%(h, m, s))

...making engineer features
load time: 0:00:02

In [ ]:
import time,os,re,csv,sys,uuid,joblib
from datetime import date
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# (from logger import update_predict_log, update_train_log)
# (from cslib import fetch_ts, engineer_features)

## model specific variables (iterate the version and note with each change)
MODEL_DIR = os.path.join("c:\\","data_ml","cs-train","models")
MODEL_VERSION = 0.1
MODEL_VERSION_NOTE = "supervised learing model for time-series"

def _model_train(df,tag,test=False):
    """
    example funtion to train model
    
    The 'test' flag when set to 'True':
        (1) subsets the data and serializes a test version
        (2) specifies that the use of the 'test' log file 

    """


    ## start timer for runtime
    time_start = time.time()
    
    X,y,dates = engineer_features(df)

    if test:
        n_samples = int(np.round(0.3 * X.shape[0]))
        subset_indices = np.random.choice(np.arange(X.shape[0]),n_samples,
                                          replace=False).astype(int)
        mask = np.in1d(np.arange(y.size),subset_indices)
        y=y[mask]
        X=X[mask]
        dates=dates[mask]
        
    ## Perform a train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                        shuffle=True, random_state=42)
    ## train a random forest model
    param_grid_rf = {
    'rf__criterion': ['mse','mae'],
    'rf__n_estimators': [10,15,20,25]
    }

    pipe_rf = Pipeline(steps=[('scaler', StandardScaler()),
                              ('rf', RandomForestRegressor())])
    
    grid = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, iid=False, n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    eval_rmse =  round(np.sqrt(mean_squared_error(y_test,y_pred)))
    
    ## retrain using all data
    grid.fit(X, y)
    model_name = re.sub("\.","_",str(MODEL_VERSION))
    if test:
        saved_model = os.path.join(MODEL_DIR,
                                   "test-{}-{}.joblib".format(tag,model_name))
        print("... saving test version of model: {}".format(saved_model))
    else:
        saved_model = os.path.join(MODEL_DIR,
                                   "sl-{}-{}.joblib".format(tag,model_name))
        print("... saving model: {}".format(saved_model))
        
    joblib.dump(grid,saved_model)

    m, s = divmod(time.time()-time_start, 60)
    h, m = divmod(m, 60)
    runtime = "%03d:%02d:%02d"%(h, m, s)

    ## update log
#    update_train_log(tag,(str(dates[0]),str(dates[-1])),{'rmse':eval_rmse},runtime,
#                     MODEL_VERSION, MODEL_VERSION_NOTE,test=True)
  

def model_train(data_dir,test=False):
    """
    funtion to train model given a df
    
    'mode' -  can be used to subset data essentially simulating a train
    """
    
    if not os.path.isdir(MODEL_DIR):
        os.mkdir(MODEL_DIR)

    if test:
        print("... test flag on")
        print("...... subseting data")
        print("...... subseting countries")
        
    ## fetch time-series formatted data
    ts_data = fetch_ts(data_dir)

    ## train a different model for each data sets
    for country,df in ts_data.items():
        
        if test and country not in ['all','united_kingdom']:
            continue
        
        _model_train(df,country,test=test)
    
def model_load(prefix='sl',data_dir=None,training=True):
    """
    example funtion to load model
    
    The prefix allows the loading of different models
    """

    if not data_dir:
        #data_dir = os.path.join("..","data","cs-train")
        data_dir = os.path.join("c:\\","data_ml","cs-train")
    
#    models = [f for f in os.listdir(os.path.join(".","models")) if re.search("sl",f)]
#    models = [f for f in os.listdir(os.path.join("c:\\","data_ml","cs-train","models")) if re.search("sl",f)]
    models = [f for f in os.listdir(os.path.join("c:\\","data_ml","cs-train","models"))]

    if len(models) == 0:
        raise Exception("Models with prefix '{}' cannot be found did you train?".format(prefix))

    all_models = {}
    for model in models:
        all_models[re.split("-",model)[1]] = joblib.load(os.path.join("c:\\","data_ml","cs-train","models",model))

    ## load data
    ts_data = fetch_ts(data_dir)
    all_data = {}
    for country, df in ts_data.items():
        X,y,dates = engineer_features(df,training=training)
        dates = np.array([str(d) for d in dates])
        all_data[country] = {"X":X,"y":y,"dates": dates}
        
    return(all_data, all_models)

def model_predict(country,year,month,day,all_models=None,test=False):
    """
    example funtion to predict from model
    """

    ## start timer for runtime
    time_start = time.time()

    ## load model if needed
    if not all_models:
        all_data,all_models = model_load(training=False)
    
    ## input checks
    if country not in all_models.keys():
        raise Exception("ERROR (model_predict) - model for country '{}' could not be found".format(country))

    for d in [year,month,day]:
        if re.search("\D",d):
            raise Exception("ERROR (model_predict) - invalid year, month or day")
    
    ## load data
    model = all_models[country]
    data = all_data[country]

    ## check date
    target_date = "{}-{}-{}".format(year,str(month).zfill(2),str(day).zfill(2))
    print(target_date)

    if target_date not in data['dates']:
        raise Exception("ERROR (model_predict) - date {} not in range {}-{}".format(target_date,
                                                                                    data['dates'][0],
                                                                                    data['dates'][-1]))
    date_indx = np.where(data['dates'] == target_date)[0][0]
    query = data['X'].iloc[[date_indx]]
    
    ## sainty check
    if data['dates'].shape[0] != data['X'].shape[0]:
        raise Exception("ERROR (model_predict) - dimensions mismatch")

    ## make prediction and gather data for log entry
    y_pred = model.predict(query)
    y_proba = None
    if 'predict_proba' in dir(model) and 'probability' in dir(model):
        if model.probability == True:
            y_proba = model.predict_proba(query)


    m, s = divmod(time.time()-time_start, 60)
    h, m = divmod(m, 60)
    runtime = "%03d:%02d:%02d"%(h, m, s)

    ## update predict log
#    update_predict_log(country,y_pred,y_proba,target_date,
#                       runtime, MODEL_VERSION, test=test)
    
    return({'y_pred':y_pred,'y_proba':y_proba})

In [ ]:
if __name__ == "__main__":

    """
    basic test procedure for model.py
    """

    ## train the model
    print("TRAINING MODELS")
    data_dir = os.path.join("c:\\","data_ml","cs-train")
    model_train(data_dir,test=False)

    ## load the model
    print("LOADING MODELS")
    all_data, all_models = model_load()
    print("... models loaded: ",",".join(all_models.keys()))

    ## test predict
    country='all'
    year='2018'
    month='01'
    day='05'
    result = model_predict(country,year,month,day)
    print(result)

TRAINING MODELS
... loading ts data from files

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-all-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-eire-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-france-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-germany-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-hong_kong-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-netherlands-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-norway-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-portugal-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-singapore-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-spain-0_1.joblib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(

... saving model: c:\data_ml\cs-train\models\sl-united_kingdom-0_1.joblib
LOADING MODELS
... loading ts data from files
... models loaded:  all,eire,france,germany,hong_kong,netherlands,norway,portugal,singapore,spain,united_kingdom
... loading ts data from files
2018-01-05
{'y_pred': array([181009.5099]), 'y_proba': None}

Case study part 3
In [ ]:

import os
import sys
import joblib
import requests
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import ensemble

if not os.path.exists("models") :
    os.mkdir("models")
#MODEL_DIR = "models"
#DATA_DIR = "data"

In [ ]:

## preprocessing pipeline
numeric_features = ['age', 'num_streams']
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                      ('scaler', StandardScaler())])

categorical_features = ['country', 'subscriber_type']
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                               ('cat', categorical_transformer, categorical_features)])


def load_aavail_data():
    df = pd.read_csv(os.path.join(DATA_DIR, r"aavail-target.csv"))

    ## pull out the target and remove uneeded columns
    _y = df.pop('is_subscriber')
    y = np.zeros(_y.size)
    y[_y==0] = 1 
    df.drop(columns=['customer_id','customer_name'],inplace=True)
    return(df,y)

In [ ]:

## load data (you may need to adjust the location of the data to match your system)
X,y = load_aavail_data()

## train test split check model performance (assumes you have already grid-searched to tune model)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
params = {'n_estimators': 100,'max_depth':2}   
clf = ensemble.RandomForestClassifier(**params)
pipe = Pipeline(steps=[('pre', preprocessor),
                       ('clf',clf)])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

## retrain using all of the data
pipe.fit(X, y)
saved_model = 'aavail-rf.joblib'
joblib.dump(pipe, os.path.join(MODEL_DIR, saved_model))

In [ ]:

%%writefile app.py

from flask import Flask, jsonify, request
import joblib
import socket
import json
import pandas as pd
import os

MODEL_DIR = "models"
DATA_DIR = "data"

app = Flask(__name__)

@app.route("/")
def hello():
    html = "<h3>Hello {name}!</h3>" \
           "<b>Hostname:</b> {hostname}<br/>"
    return html.format(name=os.getenv("NAME", "world"), hostname=socket.gethostname())

@app.route('/predict', methods=['GET','POST'])
def predict():
    
    ## input checking
    if not request.json:
        print("ERROR: API (predict): did not receive request data")
        return jsonify([])

    query = request.json
    query = pd.DataFrame(query)
    
    if len(query.shape) == 1:
         query = query.reshape(1, -1)

    y_pred = model.predict(query)
    
    return(jsonify(y_pred.tolist()))        
            
if __name__ == '__main__':
    saved_model = 'aavail-rf.joblib'
    model = joblib.load(os.path.join(MODEL_DIR, saved_model))
    app.run(host='0.0.0.0', port=8080,debug=True)

In [ ]:
%%writefile requirements.txt

cython
numpy
flask
pandas
scikit-learn

In [ ]:

%%writefile Dockerfile

# Use an official Python runtime as a parent image
FROM python:3.7.5-stretch

RUN apt-get update && apt-get install -y \
python3-dev \
build-essential    
        
# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Install any needed packages specified in requirements.txt
RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]

In [ ]:

## create some new data
X_new_data = {}
X_new_data['country'] = ['united_states','united_states','singapore','united_states','singapore']
X_new_data['age'] = [28,30,33,24,39]
X_new_data['subscriber_type'] = ['aavail_premium','aavail_basic','aavail_basic','aavail_basic','aavail_unlimited']
X_new_data['num_streams'] = [9,19,14,33,20]
X_new = pd.DataFrame(X_new_data)
X_new.head()

In [ ]:

import requests
from ast import literal_eval


## data needs to be in dict format for JSON
query = X_new.to_dict()

## test the Flask API
# port = 8080
# r = requests.post('http://0.0.0.0:{}/predict'.format(port),json=query)

## test the Docker API
port = 4000
r = requests.post('http://0.0.0.0:{}/predict'.format(port),json=query)

response = literal_eval(r.text)
print(response)

